In [6]:
import os
import ollama

In [7]:
import sys
sys.path.insert(1, '/home/user/georgy/DatasetSummarization/MetricsEvaluation')

from metrics_evaluation import MetricsEvaluator

evaluator = MetricsEvaluator()

In [8]:
import os

def load_texts(path):
    texts = []
    for filename in os.listdir(path):
        full_filename = os.path.join(path, filename)
        if os.path.isdir(full_filename):
            continue
        with open(full_filename) as f:
            texts.append(f.read())
    return texts

## Collection method

In [12]:
response_examples = [
    "Сбор данных включал генерацию ответов на различные вопросы по программированию моделью CodeLlama.",
    "В предоставленной информации нет сведений о том, как данные из датасета были собраны.",
    "Данные из датасета \"Anime Recommendations Database\" были собраны с сайта myanimelist.net, где 76 000 пользователей предоставили свои рекомендации. Датасет включает две части: одну с информацией о аниме (название, жанр, тип, количество серий, рейтинг и число участников), а другую с оценками, которые пользователи ставили различным аниме."
]

response_examples_prompt = "ПРИМЕРЫ АБЗАЦЕВ ТЕКСТА ОПИСЫВАЮЩЕГО КАК БЫЛИ СОБРАНЫ ДАННЫЕ:\n"
for i, ex in enumerate(response_examples):
    response_examples_prompt += f"Пример {i+1}:\n{ex}\n\n"

print(response_examples_prompt)

ПРИМЕРЫ АБЗАЦЕВ ТЕКСТА ОПИСЫВАЮЩЕГО КАК БЫЛИ СОБРАНЫ ДАННЫЕ:
Пример 1:
Сбор данных включал генерацию ответов на различные вопросы по программированию моделью CodeLlama.

Пример 2:
В предоставленной информации нет сведений о том, как данные из датасета были собраны.

Пример 3:
Данные из датасета "Anime Recommendations Database" были собраны с сайта myanimelist.net, где 76 000 пользователей предоставили свои рекомендации. Датасет включает две части: одну с информацией о аниме (название, жанр, тип, количество серий, рейтинг и число участников), а другую с оценками, которые пользователи ставили различным аниме.




In [13]:
instruction = "Используй информацию из секций, обозначенных ЗАГЛАВНЫМИ буквами, чтобы составить один абзац текста описывающий как данные из датасета были собраны. "
instruction += "Если в секциях выше нет информации о том, как данные были собраны, то просто сообщи об этом. "
instruction += "Свой ответ составь на русском языке, формулируя ответ похожим на ПРИМЕРЫ АБЗАЦЕВ ТЕКСТА ОПИСЫВАЮЩЕГО КАК БЫЛИ СОБРАНЫ ДАННЫЕ. "

instruction

'Используй информацию из секций, обозначенных ЗАГЛАВНЫМИ буквами, чтобы составить один абзац текста описывающий как данные из датасета были собраны. Если в секциях выше нет информации о том, как данные были собраны, то просто сообщи об этом. Свой ответ составь на русском языке, формулируя ответ похожим на ПРИМЕРЫ АБЗАЦЕВ ТЕКСТА ОПИСЫВАЮЩЕГО КАК БЫЛИ СОБРАНЫ ДАННЫЕ. '

In [18]:
dir = 'CollectionMethodPrompts'

for filename in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, filename)):
        continue
    # print(filename)
    with open(os.path.join(dir, filename), 'r') as f:
        prompt = f.read()
    paragraphs = prompt.split('\n\n')
    paragraphs = paragraphs[:-1]
    dataset_prompt = '\n\n'.join(paragraphs)
    prompt = f"{dataset_prompt}\n\n{response_examples_prompt}\n\n{instruction}"
    with open(os.path.join(dir, filename), 'w') as f:
        f.write(prompt)
    # break

In [19]:
from tqdm import tqdm

output_dir = 'CollectionMethodResponses'

def generate(prompt):
    result = ollama.generate(model='saiga-llama3-t0', prompt=prompt)
    response = result['response']
    response = response.strip()
    return response

for filename in tqdm(os.listdir(dir)):
    if os.path.isdir(os.path.join(dir, filename)):
        continue
    with open(os.path.join(dir, filename), 'r') as f:
        prompt = f.read()
    response = generate(prompt)
    with open(os.path.join(output_dir, filename), 'w') as f:
        f.write(response)
    # break

100%|██████████| 68/68 [01:25<00:00,  1.26s/it]


In [21]:
common_folder = os.path.join('/', 'home', 'user', 'georgy', 'DatasetSummarization', 'DatasetSummarization', 'KaggleMigration')
responses_folder = os.path.join(common_folder, 'Experiments', 'FewshotPrompting', 'CollectionMethodResponses')
references_folder = os.path.join(common_folder, 'KaggleMigrationDataset', 'CollectionMethodReferences')

references = load_texts(references_folder)
responses = load_texts(responses_folder)

evaluator.calculate_scores(responses, references)

/root/miniconda3/envs/ncs_georgy/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/miniconda3/envs/ncs_georgy/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/miniconda3/envs/ncs_georgy/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider usi

{'bleu': 1.0329095656883166e-231,
 'rouge': {'precision': 0.15711825487944892,
  'recall': 0.1855721393034826,
  'f1': 0.16018285197389676},
 'meteor': 0.15206631318427716,
 'bertscore': {'precision': 0.6605818867683411,
  'recall': 0.7059652805328369,
  'f1': 0.681296706199646}}

## Dataset Structure

In [35]:
response_examples = [
    'Датасет содержит подробную информацию о 52,000 анимационных фильмах, собранную с помощью TMDB API. Основной файл датасета называется "Animation_Movies.csv" и включает различные атрибуты для каждого фильма, такие как "id", "title" (название), "vote_average" (средняя оценка), "vote_count" (количество голосов), "status" (статус), "release_date" (дата выпуска), "revenue" (доход), "runtime" (длительность), "adult" (для взрослых), "backdrop_path" (путь к фоновому изображению), "budget" (бюджет), "homepage" (домашняя страница), "imdb_id", "original_language" (оригинальный язык), "original_title" (оригинальное название), "overview" (обзор), "popularity" (популярность), "poster_path" (путь к постеру), "tagline" (слоган), "genres" (жанры), "production_companies" (компании-производители), "production_countries" (страны-производители) и "spoken_languages" (разговорные языки).',
    'Датасет содержит данные для классификации и анализа DDoS-атак. Один из основных файлов датасета называется `train_dataset.csv`. Данные включают различные поля, такие как тип протокола, служба, состояние соединения, количество пакетов, байтов и другие сетевые параметры, заканчивающиеся меткой, обозначающей тип трафика (например, \'normal\' или \'snmpgetattack\').',
    'Датасет содержит информацию о различных автомобилях. Он включает следующие столбцы: `Unnamed: 0`, `brand` (марка автомобиля), `model` (модель автомобиля), `color` (цвет), `registration_date` (дата регистрации), `year` (год выпуска), `price_in_euro` (цена в евро), `power_kw` (мощность в киловаттах), `power_ps` (мощность в лошадиных силах), `transmission_type` (тип трансмиссии), `fuel_type` (тип топлива), `fuel_consumption_l_100km` (расход топлива в литрах на 100 км), `fuel_consumption_g_km` (расход топлива в граммах на км), `mileage_in_km` (пробег в км), и `offer_description` (описание предложения). Эти данные представлены в формате CSV и обеспечивают всестороннюю информацию о характеристиках автомобилей, включая их технические детали и состояние.',
]

response_examples_prompt = ""
for i, ex in enumerate(response_examples):
    response_examples_prompt += f"Пример {i+1}:\n{ex}\n\n"

print(response_examples_prompt)

Пример 1:
Датасет содержит подробную информацию о 52,000 анимационных фильмах, собранную с помощью TMDB API. Основной файл датасета называется "Animation_Movies.csv" и включает различные атрибуты для каждого фильма, такие как "id", "title" (название), "vote_average" (средняя оценка), "vote_count" (количество голосов), "status" (статус), "release_date" (дата выпуска), "revenue" (доход), "runtime" (длительность), "adult" (для взрослых), "backdrop_path" (путь к фоновому изображению), "budget" (бюджет), "homepage" (домашняя страница), "imdb_id", "original_language" (оригинальный язык), "original_title" (оригинальное название), "overview" (обзор), "popularity" (популярность), "poster_path" (путь к постеру), "tagline" (слоган), "genres" (жанры), "production_companies" (компании-производители), "production_countries" (страны-производители) и "spoken_languages" (разговорные языки).

Пример 2:
Датасет содержит данные для классификации и анализа DDoS-атак. Один из основных файлов датасета называ

In [40]:
instruction = "Используй информацию из секций выше, обозначенных ЗАГЛАВНЫМИ буквами, чтобы составить один абзац текста описывающий структуру датасета. "
instruction += "Свой ответ составь на русском языке. Вот примеры ответов:\n"
instruction += response_examples_prompt

print(instruction)

Используй информацию из секций выше, обозначенных ЗАГЛАВНЫМИ буквами, чтобы составить один абзац текста описывающий структуру датасета. Свой ответ составь на русском языке. Вот примеры ответов:
Пример 1:
Датасет содержит подробную информацию о 52,000 анимационных фильмах, собранную с помощью TMDB API. Основной файл датасета называется "Animation_Movies.csv" и включает различные атрибуты для каждого фильма, такие как "id", "title" (название), "vote_average" (средняя оценка), "vote_count" (количество голосов), "status" (статус), "release_date" (дата выпуска), "revenue" (доход), "runtime" (длительность), "adult" (для взрослых), "backdrop_path" (путь к фоновому изображению), "budget" (бюджет), "homepage" (домашняя страница), "imdb_id", "original_language" (оригинальный язык), "original_title" (оригинальное название), "overview" (обзор), "popularity" (популярность), "poster_path" (путь к постеру), "tagline" (слоган), "genres" (жанры), "production_companies" (компании-производители), "produc

In [41]:
dir = 'DatasetStructurePrompts'

for filename in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, filename)):
        continue
    with open(os.path.join(dir, filename), 'r') as f:
        prompt = f.read()
    paragraphs = prompt.split('\n\n')
    paragraphs = paragraphs[:-1]
    dataset_prompt = '\n\n'.join(paragraphs)
    prompt = f"{dataset_prompt}\n\n{instruction}"
    with open(os.path.join(dir, filename), 'w') as f:
        f.write(prompt)
    print(filename)
    break

moodify-dataset.txt


In [42]:
from tqdm import tqdm

output_dir = 'DatasetStructureResponses'

def generate(prompt):
    result = ollama.generate(model='saiga-llama3-t0', prompt=prompt)
    response = result['response']
    response = response.strip()
    return response

for filename in tqdm(os.listdir(dir)):
    if os.path.isdir(os.path.join(dir, filename)):
        continue
    with open(os.path.join(dir, filename), 'r') as f:
        prompt = f.read()
    response = generate(prompt)
    with open(os.path.join(output_dir, filename), 'w') as f:
        f.write(response)
    break

  0%|          | 0/68 [00:03<?, ?it/s]


In [95]:
common_folder = os.path.join('/', 'home', 'user', 'georgy', 'DatasetSummarization', 'DatasetSummarization', 'KaggleMigration')
responses_folder = os.path.join(common_folder, 'Experiments', 'PromptRefinement', 'DatasetStructureResponses')
references_folder = os.path.join(common_folder, 'KaggleMigrationDataset', 'DatasetStructureReferences')

references = load_texts(references_folder)
responses = load_texts(responses_folder)

evaluator.calculate_scores(responses, references)

/root/miniconda3/envs/ncs_georgy/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/miniconda3/envs/ncs_georgy/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


{'bleu': 9.361888126935942e-156,
 'rouge': {'precision': 0.5055759642513374,
  'recall': 0.19833173918009975,
  'f1': 0.23441545982577153},
 'meteor': 0.1752210285223062,
 'bertscore': {'precision': 0.7285274267196655,
  'recall': 0.6587602496147156,
  'f1': 0.6912165880203247}}

## Potential Uses

In [99]:
instruction = "Cоставь один абзац текста описывающий возможные варианты использования датасета. "
instruction += "Не упоминай ID датасета, заголовок и подзаголовок. "
instruction += "Не рассказывай как был собран датасет. "
instruction += "Не говори о ключевых словах датасета. "
instruction += "Не упоминай себя и автоматического ассистента Сайга. "
instruction += "Сформулируй ответ в одном абзаце на русском языке."

instruction

'Cоставь один абзац текста описывающий возможные варианты использования датасета. Не упоминай ID датасета, заголовок и подзаголовок. Не рассказывай как был собран датасет. Не говори о ключевых словах датасета. Не упоминай себя и автоматического ассистента Сайга. Сформулируй ответ в одном абзаце на русском языке.'

In [100]:
dir = 'PotentialUsesPrompts'

for filename in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, filename)):
        continue
    with open(os.path.join(dir, filename), 'r') as f:
        prompt = f.read()
    paragraphs = prompt.split('\n\n')
    paragraphs = paragraphs[:-1]
    dataset_prompt = '\n\n'.join(paragraphs)
    prompt = f"{dataset_prompt}\n\n{instruction}"
    with open(os.path.join(dir, filename), 'w') as f:
        f.write(prompt)

In [101]:
from tqdm import tqdm

output_dir = 'PotentialUsesResponses'

def generate(prompt):
    result = ollama.generate(model='saiga-llama3-t0', prompt=prompt)
    response = result['response']
    response = response.strip()
    return response

for filename in tqdm(os.listdir(dir)):
    if os.path.isdir(os.path.join(dir, filename)):
        continue
    with open(os.path.join(dir, filename), 'r') as f:
        prompt = f.read()
    response = generate(prompt)
    with open(os.path.join(output_dir, filename), 'w') as f:
        f.write(response)

100%|██████████| 68/68 [02:06<00:00,  1.86s/it]


In [102]:
common_folder = os.path.join('/', 'home', 'user', 'georgy', 'DatasetSummarization', 'DatasetSummarization', 'KaggleMigration')
responses_folder = os.path.join(common_folder, 'Experiments', 'PromptRefinement', 'PotentialUsesResponses')
references_folder = os.path.join(common_folder, 'KaggleMigrationDataset', 'PotentialUsesReferences')

references = load_texts(references_folder)
responses = load_texts(responses_folder)

evaluator.calculate_scores(responses, references)

{'bleu': 9.710120333628321e-232,
 'rouge': {'precision': 0.014925373134328358,
  'recall': 0.007462686567164179,
  'f1': 0.009950248756218905},
 'meteor': 0.18346541065719804,
 'bertscore': {'precision': 0.7410532236099243,
  'recall': 0.7334495782852173,
  'f1': 0.7370179295539856}}